In [16]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [17]:
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("20190801", "20220731")

In [18]:
AUTH_KEY = 'aU1w0v96iPjc398MIHS82Q6/fOhoZfeIg4ugUBHFagbKALzb4jcV66srdi8uG9TcW26oM4zOy84/F6ZgRnV/4Q=='

In [19]:
def getWeather(date, key):
    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    params = {
        'serviceKey' : key,
        'dataType' : 'JSON',
        'startDt' : date,
        'endDt' : date,
        'stnIds' : '184',
        'dataCd' : 'ASOS',
        'dateCd' : 'DAY'
    }

    res = requests.get(url,params=params)
    dic = json.loads(res.text)
    item =  dic['response']['body']['items']['item']

    return item

In [20]:
weather_list =[]
for date in dates:
    weather_list.extend(getWeather(date, AUTH_KEY))

In [21]:
weather_df = pd.DataFrame(weather_list, columns=['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'minTaHrmt', 'maxTa', 'maxTaHrmt', 'mi10MaxRn', \
  'mi10MaxRnHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxInsWsWd', 'maxInsWsHrmt', \
  'maxWs', 'maxWsWd', 'maxWsHrmt', 'avgWs', 'hr24SumRws', 'maxWd', 'avgTd', 'minRhm', 'minRhmHrmt', \
  'avgRhm', 'avgPv', 'avgPa', 'maxPs', 'maxPsHrmt', 'minPs', 'minPsHrmt', 'avgPs', 'ssDur', 'sumSsHr', \
  'hr1MaxIcsrHrmt', 'hr1MaxIcsr', 'sumGsr', 'ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt', 'sumDpthFhsc', \
  'avgTca', 'avgLmac', 'avgTs', 'minTg', 'avgCm5Te', 'avgCm10Te', 'avgCm20Te', 'avgCm30Te', 'avgM05Te', \
  'avgM10Te', 'avgM15Te', 'avgM30Te', 'avgM50Te', 'sumLrgEv', 'sumSmlEv', 'n99Rn', 'iscs', 'sumFogDur']
)

In [22]:
weather_df.to_csv('./weather.csv', encoding = 'utf-8-sig')

tm : 시간  
avgTa : 평균 기온  
minTa : 최저 기온  
maxTa : 최고 기온  
sumRnDur : 강수 계속시간  
sumRn : 일강수량  
avgWs : 평균 풍속  
ddMes : 일 최심적설  
avgTca : 평균 전운량  
sumFogDur : 안개 계속 시간  

In [ ]:
temp_weather_df = weather_df.copy()

In [25]:
temp_weather_df = temp_weather_df[['tm', 'avgTa', 'minTa', 'maxTa', 'sumRnDur', 'sumRn', 'avgWs', 'ddMes', 'avgTca', 'sumFogDur']]
temp_weather_df = temp_weather_df.replace('', 0)
temp_weather_df = temp_weather_df.apply(pd.to_numeric)

In [32]:
temp_weather_df['tm'] = pd.to_datetime(temp_weather_df['tm'])
temp_weather_df['month'] = temp_weather_df['tm'].dt.month
temp_weather_df['day'] = temp_weather_df['tm'].dt.day
temp_weather_df = temp_weather_df.drop('tm', axis = 1)

In [64]:
grouped_weather_df = temp_weather_df.groupby(['month', 'day'])
grouped_weather_df_mean = pd.DataFrame(grouped_weather_df.mean()).reset_index()
grouped_weather_df_mean

In [71]:
grouped_weather_df_mean.to_csv('./weather_last3.csv', encoding = 'utf-8-sig')